## Installing required external libraries/packages

We are using python, and it's package manager pip to do so.

We also run the command ```lscpu``` to learn more about the computer that is running this jupyer notebook.

In [1]:
!pip install --user biopython channels bokeh==2.3.0 panel==0.11.0
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   40 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              1
Core(s) per socket:              1
Socket(s):                       8
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           63
Model name:                      Intel(R) Xeon(R) CPU E5-2630 v3 @ 2.40GHz
Stepping:                        2
CPU MHz:                         2399.996
BogoMIPS:                        4799.99
Virtualization:                  VT-x
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       256 KiB
L1i cache:                       256 KiB
L2 cache:                        32 MiB
L3 cache:                        128 MiB


## Download a sequence alignment program (muscle)

There are many different sequence alignment programs. Muscle is fast and works well with highly similar sequences.

In [2]:
!wget http://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86linux64.tar.gz
!tar -zxvf muscle3.8.31_i86linux64.tar.gz
!mv muscle3.8.31_i86linux64 muscle

--2021-03-25 23:50:02--  http://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86linux64.tar.gz
Resolving www.drive5.com (www.drive5.com)... 199.195.116.69
Connecting to www.drive5.com (www.drive5.com)|199.195.116.69|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 487906 (476K) [application/x-gzip]
Saving to: ‘muscle3.8.31_i86linux64.tar.gz.2’

muscle3.8.31_i86lin 100%[===================>] 476.47K  1007KB/s    in 0.5s    

2021-03-25 23:50:03 (1007 KB/s) - ‘muscle3.8.31_i86linux64.tar.gz.2’ saved [487906/487906]

muscle3.8.31_i86linux64


### Now we test to see if the program was downloaded and installed correctly.

In [3]:
!./muscle


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.


Basic usage

    muscle -in <inputfile> -out <outputfile>

Common options (for a complete list please see the User Guide):

    -in <inputfile>    Input file in FASTA format (default stdin)
    -out <outputfile>  Output alignment in FASTA format (default stdout)
    -diags             Find diagonals (faster for similar sequences)
    -maxiters <n>      Maximum number of iterations (integer, default 16)
    -maxhours <h>      Maximum time to iterate in hours (default no limit)
    -html              Write output in HTML format (default FASTA)
    -msf               Write output in GCG MSF format (default FASTA)
    -clw               Write output in CLUSTALW format (default FASTA)
    -clwstrict         As -clw, with 'CLUSTAL W (1.81)' header
    -log[a] <logfile>  Log to file (append if -loga, overwrite if -log)
   

### Once all our libraries/packages are obtained through pip, we can import them to be able to use them.

In [25]:
import os, io, random
import string
import numpy as np
import subprocess
import re

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO
from Bio import GenBank

import panel as pn
import panel.widgets as pnw
pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d, CustomJS, RangeSlider
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot
from bokeh.events import Tap

from collections import Counter

## Helper Functions

view_alignment helps visualize the alignment of the sequences.

import_virus will read a downloaded GenBank file and import it.

codon_dictionary helps us translate a dna sequence into a protein sequence.

In [5]:
def view_alignment(aln, fontsize="9pt", plot_width=800, isDNA=True, sizing_mode='fixed', row_height=10, annot=None, labels=None):
    """Bokeh sequence alignment view"""
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]
    if len(seqs) <= 1:
        p = plot_empty('needs at least two sequences',plot_width)
        return p
    #ids=range(len(seqs))
    text = [i for s in list(seqs) for i in s]
    if isDNA:
        colors = get_dna_colors(seqs)
    else:
        colors = get_protein_colors(seqs)
    x=[]
    l = len(aln)
    for i in range(aln.get_alignment_length()):
        a = aln[:,i]
        res = Counter(a)
        del(res['-'])
        x.append(max(res.values())/l)
        #print (a,res,max(res.values())/l)
    cons = x
    N = len(seqs[0])
    S = len(seqs)
    width=.4

    x = np.arange(1, N+1)
    y = np.arange(0,S,1)
    #print (y[:20])
    xx, yy = np.meshgrid(x, y)
    gx = xx.ravel()
    gy = yy.flatten()
    recty = gy+.5
    h= 1/S

    #print (text)
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs) * row_height + 50
    x_range = Range1d(0,N+1, bounds='auto')
    L=100
    if len(seqs[0])<100:
        L=len(seqs[0])
    view_range = (0,L)
    viewlen = view_range[1]-view_range[0]
    tools="xpan, xwheel_zoom, tap, reset, save"

    #preview sequence view (no text)
    p = figure(title=None, plot_width=plot_width, plot_height=S*2+25, x_range=x_range, y_range=(0,S), tools=tools,
                    min_border=0, toolbar_location=None, sizing_mode=sizing_mode)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors", line_color=None, fill_alpha=0.4)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False

    #full sequence text view
    p1 = figure(title=None, plot_width=plot_width, plot_height=plot_height, x_range=view_range, y_range=ids, tools="xpan,reset",
                    min_border=0, toolbar_location='below')#, lod_factor=1)
    seqtext = Text(x="x", y="y", text="text", text_align='center',text_color="black", text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors", line_color=None, fill_alpha=0.5)
    p1.add_glyph(source, rects)
    p1.add_glyph(source, seqtext)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0
    p1.toolbar.logo = None

    source2 = ColumnDataSource(dict(x=x, cons=cons))

    p3 = figure(title=None, plot_width=plot_width, plot_height=50, x_range=p1.x_range, y_range=(Range1d(min(cons),.5)), tools="xpan")
    rects2 = Rect(x="x", y=0,  width=1, height="cons", fill_color="gray", line_color=None, fill_alpha=0.7)
    p3.add_glyph(source2, rects2)

    p3.xaxis.visible = False
    p3.yaxis.visible = False
    p3.grid.visible = False
    p3.background_fill_color = "beige"

    #callback for click
    jscode="""
        row = parseInt(cb_obj.y);
        console.log(row);
    """
    clicked = CustomJS(args=dict(source=source),code=jscode)
    p1.js_on_event(Tap, clicked)

    #callback for slider move
    jscode="""
        var start = cb_obj.value[0];
        var end = cb_obj.value[1];
        x_range.setv({"start": start, "end": end})
        text.text_font_size=fontsize+"pt";
    """
    callback = CustomJS(
        args=dict(x_range=p1.x_range,text=seqtext), code=jscode)
    slider = RangeSlider(start=0, end=N, value=(0,L), step=10, min_width=600, max_width=1200, sizing_mode='stretch_width')
    slider.js_on_change('value_throttled', callback)

    #callback for plot drag
    jscode="""
        start = parseInt(range.start);
        end = parseInt(range.end);
        slider.value[0] = start;
    """
    callback = CustomJS(args=dict(slider=slider, range=p1.x_range),
                                  code=jscode)
    p.x_range.js_on_change('start', callback)
    p = gridplot([[p],[p3],[p1],[slider]], toolbar_location=None, sizing_mode=sizing_mode)
    return p

def get_dna_colors(seqs):
    """make colors for bases in sequence"""
    text = [i for s in list(seqs) for i in s]
    clrs =  {'A':'red','T':'green','G':'orange','C':'blue','-':'white', 'N':'gray'}
    colors = [clrs[i] for i in text]
    return colors

def get_protein_colors(seqs):
    text = [i for s in list(seqs) for i in s]
    clrs =  {'D':'#1f77b4','E':'#aec7e8','S':'#ff7f0e','T':'#ffbb78','Y':'#2ca02c',
             'A':'#98df8a','V':'#d62728','L':'#ff9896','I':'#9467bd','P':'#c5b0d5',
             'F':'#8c564b','G':'#c49c94','C':'#e377c2','M':'#f7b6d2','K':'#7f7f7f',
             'W':'#c7c7c7','N':'#bcbd22','Q':'#dbdb8d','H':'#17becf','R':'#9edae5',
             '-':'white','X':'black'}
    colors = [clrs[i] for i in text]
    return colors

def muscle_alignment(seqs, filename):
    """Align 2 sequences with muscle"""
    SeqIO.write(seqs, filename, "fasta")
    name = os.path.splitext(filename)[0]
    out_file = name+'.txt'
    muscle_result = subprocess.check_output(["./muscle", "-in", filename, "-out", out_file])
    return

In [26]:
def import_virus(filename):
    with open(f"{filename}.gb") as handle:
        record = GenBank.read(handle)
    print(f"Importing {filename} virus: {record.accession}")
    dna_seqs = dict()
    protein_seqs = dict()
    virus_record = SeqRecord(
        Seq(record.sequence),
        id=record.accession[0],
        name=filename,
        description="SARS-CoV-2 (HCoV-19) Genome"
    )

    for f in record.features:
        foundKey = ""
        for q in f.qualifiers:
            if f.key == "gene" and q.key == "/gene=":
                str_start, _, str_end = f.location.partition("..")
                s_gene_start = int(re.sub("[^0-9]", "", str_start))
                s_gene_end = int(re.sub("[^0-9]", "", str_start))
                dna_seqs[q.value[1:-1].lower()] = (s_gene_start, s_gene_end)
            if f.key == "CDS" and q.key == "/gene=":
                foundKey = q.value[1:-1].lower()
                protein_seqs[foundKey] = ""
            if foundKey and q.key == "/translation=":
                protein_seqs[foundKey] = q.value[1:-1]
                foundKey = ""
                #DNA contains TAA for stop codon which does not appear in AA sequence
    return virus_record, dna_seqs, protein_seqs

In [7]:
def example_aligner():
    title = pn.pane.Markdown('## Sequence aligner')
    load_btn = pn.widgets.FileInput()
    aln_btn = pn.widgets.Button(name='align',width=100,button_type='primary')
    randomseq_btn = pn.widgets.Button(name='random seqs',width=100,button_type='primary')
    translateseq_btn = pn.widgets.Button(name='translate seqs',width=100,button_type='primary')
    numseqs_input = pn.widgets.IntSlider(name='sequences',start=2,end=4,value=4,width=200)
    length_input = pn.widgets.IntSlider(name='length',start=10,end=330,value=50,width=200)

    seq_pane = pn.pane.Str(name='sequences',height=200, width=1200)
    result = pn.pane.Str("empty",width=600)
    untranslated = pn.pane.Str("empty")
    translation_toogle = pn.pane.Str("empty")
    bokeh_pane = pn.pane.Bokeh(height=500,margin=10, width=1200)

    def make_seq(length=40):    
        return ''.join([random.choice(['A','C','T','G']) for i in range(length)])

    def mutate_seq(seq):
        """mutate a sequence randomly"""
        seq = list(seq)
        pos = np.random.randint(1,len(seq),6)    
        for i in pos:
            seq[i] = random.choice(['A','C','T','G'])
        return ''.join(seq)

    def create_sequences(event):
        #creates a set of sequences using widget values
        s=''
        seqlen = length_input.value
        startseq = make_seq(seqlen)
        num = numseqs_input.value
        for i in range(num):
            seq = mutate_seq(startseq)
            name = f"Sequence{i}"
            s+='>%s\n' %name + seq+'\n'
        seq_pane.object = s
        untranslated.object = s
        translation_toogle.object = 'untranslated'
        return

    def align(event):
        #this function does the alignment using the textinput values    
        s = seq_pane.object
        sequences = SeqIO.parse(io.StringIO(s),format='fasta')
        sequences = [rec for rec in sequences]
        muscle_alignment(sequences, "temp.fa")
        aln = AlignIO.read("temp.txt", 'fasta')
        #the result widget is then updated    
        result.object = aln
        #aligned = [rec.seq for rec in (aln)]
        bokeh_pane.object = view_alignment(aln,fontsize="7pt",plot_width=1200,isDNA=bool(translation_toogle.object))
        return

    def translate_sequences(event):
        #naively translates DNA into protein (removes stop codons)
        try:
            s = seq_pane.object
            sequences = SeqIO.parse(io.StringIO(s),format='fasta')
            output_s = ''
            for rec in sequences:
                name = rec.id
                output_s += f">{name}\n{rec.translate(stop_symbol='')._seq}\n"
            seq_pane.object = output_s
            translation_toogle.object = ''
        except:
            seq_pane.object = untranslated.object
            translation_toogle.object = 'untranslated'
        return


    aln_btn.param.watch(align, 'clicks')
    randomseq_btn.param.watch(create_sequences, 'clicks')
    translateseq_btn.param.watch(translate_sequences, 'clicks')

    top = pn.Row(randomseq_btn, translateseq_btn, aln_btn, length_input, numseqs_input)
    middle = pn.Row(seq_pane, sizing_mode='fixed')
    bottom = pn.Row(bokeh_pane, sizing_mode='fixed')
    return pn.Column(title,top,middle,bottom)

In [8]:
def covid_aligner(what_to_align):
    what_to_align = what_to_align.lower()
    isDNA = True
    if what_to_align == "genome":
        seqs = [reference_record, choice_record]
    elif what_to_align.startswith("dna"):
        part = what_to_align.split(':')[-1].strip()
        reference_part_seq = reference_record[reference_dna_seqs[part][0]-1:reference_dna_seqs[part][1]]
        choice_part_seq = choice_record[choice_dna_seqs[part][0]-1:choice_dna_seqs[part][1]]
        seqs = [reference_part_seq, choice_part_seq]
    elif what_to_align.startswith("protein"):
        part = what_to_align.split(':')[-1].strip()
        reference_part_seq = SeqRecord(Seq(reference_protein_seqs[part]), id=f"reference_{part}", description="Reference protein")
        choice_part_seq = SeqRecord(Seq(choice_protein_seqs[part]), id=f"chosen_{part}", description="Chosen protein")
        seqs = [reference_part_seq, choice_part_seq]
        isDNA = False
    title = pn.pane.Markdown("## Sequence aligner")
    bokeh_pane2 = pn.pane.Bokeh(height=500,margin=5)

    muscle_alignment(seqs, "hcov-19.fa")
    aln = AlignIO.read("hcov-19.txt", 'fasta')
    bokeh_pane2.object = view_alignment(aln,fontsize="7pt",plot_width=1200,isDNA=isDNA)

    top2 = pn.Row(title, sizing_mode='stretch_height')
    bottom2 = pn.Row(bokeh_pane2, sizing_mode='stretch_height')
    return pn.Column(top2,bottom2)

In [9]:
def get_differences():
    differences = []
    with open("hcov-19.txt") as aligned_file:
        sequences = SeqIO.parse(aligned_file, format='fasta')
        seq1 = next(sequences)
        seq2 = next(sequences)
        if len(seq1) != len(seq2):
            print("error")
        else:
          for i in range(len(seq1)):
              if seq1[i] != seq2[i]:
                  differences.append(i+1)
    print(differences)

Central dogma:           |  RNA VS DNA:
:---------------------:|:---------------------:
<img src="https://faculty.lsu.edu/avourekas/images/centraldogmaexpandedb.png" width="600px"/> | ![DNA RNA Differences](https://cm.jefferson.edu/wordpress/wp-content/uploads/2014/01/pasted-graphic-11.jpg "DNA RNA Differences")



Codon chart:           |  Codon table:
:-----------------------:|:-----------------------:
![Codon chart](https://upload.wikimedia.org/wikipedia/commons/7/70/Aminoacids_table.svg "Codon chart") | ![Codon table](https://cdn.kastatic.org/ka-perseus-images/f5de6355003ee322782b26404ef0733a1d1a61b0.png "Codon table")

In [10]:
codon_dictionary = {
    'GAT': 'D', 'GAA': 'E', 'TCT': 'S', 'TCA': 'S', 'AGT': 'S', 'ACT': 'T', 'ACA': 'T', 'TAT': 'Y', 'GCT': 'A', 'GCA': 'A', 'GTT': 'V', 'GTA': 'V', 'TTA': 'L', 'CTT': 'L', 'CTA': 'L', 'CCT': 'P', 'CCA': 'P', 'TTT': 'F', 'GGT': 'G', 'GGA': 'G', 'TGT': 'C', 'AAA': 'K', 'AAT': 'N', 'CAA': 'Q', 'CAT': 'H', 'CGT': 'R', 'CGA': 'R', 'AGA': 'R', 'ATT': 'I', 'ATA': 'I', 'TGG': 'W',
    'GAC': 'D', 'GAG': 'E', 'TCC': 'S', 'TCG': 'S', 'AGC': 'S', 'ACC': 'T', 'ACG': 'T', 'TAC': 'Y', 'GCC': 'A', 'GCG': 'A', 'GTC': 'V', 'GTG': 'V', 'TTG': 'L', 'CTC': 'L', 'CTG': 'L', 'CCC': 'P', 'CCG': 'P', 'TTC': 'F', 'GGC': 'G', 'GGG': 'G', 'TGC': 'C', 'AAG': 'K', 'AAC': 'N', 'CAG': 'Q', 'CAC': 'H', 'CGC': 'R', 'CGG': 'R', 'AGG': 'R', 'ATC': 'I', 'ATG': 'M'}

# Let's start with an interactive example for aligning sequences

In [11]:
example_aligner()

Column
    [0] Markdown(str)
    [1] Row
        [0] Button(button_type='primary', name='random seqs', width=100)
        [1] Button(button_type='primary', name='translate seqs', width=100)
        [2] Button(button_type='primary', name='align', width=100)
        [3] IntSlider(end=330, name='length', start=10, value=50, value_throttled=50, width=200)
        [4] IntSlider(end=4, name='sequences', start=2, value=4, value_throttled=4, width=200)
    [2] Row(sizing_mode='fixed')
        [0] Str(None, height=200, name='sequences', sizing_mode='fixed', width=1200)
    [3] Row(sizing_mode='fixed')
        [0] Bokeh(None, height=500, margin=10, sizing_mode='fixed', width=1200)

## Now we put it into practice with real data by downloading the reference SARS-CoV-2 (hCoV-19) sequence

In [12]:
#https://www.ncbi.nlm.nih.gov/nuccore/MN908947.3/
!wget -O reference.gb "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=gbwithparts&id=1798172431&withparts=on"
reference_record, reference_dna_seqs, reference_protein_seqs = import_virus("reference")

--2021-03-25 23:50:10--  https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=gbwithparts&id=1798172431&withparts=on
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘reference.gb’

reference.gb            [ <=>                ]  55.48K  --.-KB/s    in 0.09s   

2021-03-25 23:50:11 (601 KB/s) - ‘reference.gb’ saved [56807]

Importing reference virus: ['MN908947']


## We can compare it to a SARS-CoV-2 virus of our choice

You can find them [here (NCBI website)](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=Wuhan%20seafood%20market%20pneumonia%20virus,%20taxid:2697049&utm_source=gquery&utm_medium=referral&utm_campaign=COVID-19)

In [27]:
#Find your genome of choice at NCBI and insert it below
id_of_choice = "1938534781"
#Replace id below with the GI in the summary
!wget -O our_choice.gb "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=genbank&id=$id_of_choice&conwithfeat=on&withparts=on&hide-cdd=on"
choice_record, choice_dna_seqs, choice_protein_seqs = import_virus("our_choice")

--2021-03-26 00:21:37--  https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=genbank&id=1938534781&conwithfeat=on&withparts=on&hide-cdd=on
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘our_choice.gb’

our_choice.gb           [ <=>                ]  65.06K   414KB/s    in 0.2s    

2021-03-26 00:21:37 (414 KB/s) - ‘our_choice.gb’ saved [66625]

Importing our_choice virus: ['MW309425']


In [28]:
#Choose only one from below! (Therefore, two lines must be green with the selected one being black)
#what_to_align = "Genome"
#what_to_align = "DNA_part:" + "S"
what_to_align = "Protein_part:" + "S"
#-----------------------------------------------------------------------------------------------#

covid_aligner(what_to_align)

Column
    [0] Row(sizing_mode='stretch_height')
        [0] Markdown(str)
    [1] Row(sizing_mode='stretch_height')
        [0] Bokeh(GridBox, height=500)

If you are having trouble finding differences, you can use the cell below to help you locate them.

In [29]:
get_differences()

[614]
